In [1]:
from python_lib.ecapa_classifier import *
import torch
import numpy as np
import os
from python_lib.saveasfile import *

c:\Users\chiny\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\chiny\anaconda3\envs\pytorch\lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")
SpeechBrain could not find any working torchaudio backend. Audio files may fail to load. Follow this link for instructions and troubleshooting: https://pytorch.org/audio/stable/index.html
SpeechBrain could not find any working torchaudio backend. Audio files may fail to load. Follow this link for instructions and troubleshooting: https://pytorch.org/audio/stable/index.html


In [2]:
model = AttentiveStatisticsPooling(16,128).to("cuda")

BlockSave(model.return_layers(), "aspblock", "ECAPAweights")

In [3]:
input_feats = torch.rand([1,16,64]).to("cuda")
input_feats_np = input_feats.cpu().detach().numpy()
input_feats_np = np.reshape(input_feats_np, (16,64))
print(input_feats_np)
dim = input_feats_np.shape
print(dim)

flatten_inputs = input_feats_np.flatten()
with open(os.path.join("ECAPAweights", f"ecapainput_8x64.bin"), "wb") as f:
            # Write the dimensions down
            f.write(np.array(dim, dtype=np.int32).tobytes())
            # Write the flatten bias down
            f.write(flatten_inputs.tobytes())

[[0.92080986 0.94263554 0.76715916 ... 0.3460167  0.7671066  0.09391695]
 [0.36231142 0.6570243  0.8429503  ... 0.41927123 0.7319405  0.5816679 ]
 [0.86564565 0.10518014 0.0736413  ... 0.01690978 0.13004255 0.70561683]
 ...
 [0.7154462  0.83649814 0.8597458  ... 0.6584436  0.65377873 0.44357592]
 [0.15143669 0.0156619  0.48285717 ... 0.64426965 0.04532176 0.32158285]
 [0.17286223 0.5211401  0.78973126 ... 0.14768445 0.6469075  0.26730663]]
(16, 64)


In [4]:
model.eval()

x = model(input_feats, lengths = torch.ones((1))*0.5)
x